In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.decomposition import NMF
import numpy as np
import pickle 
import warnings  
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

import jieba
import jieba.posseg as pseg

In [45]:
history

# 作业一：为 content 中的内容分词，请替换下行中的 ...
df_orig['content'] = (df_orig['content']).apply(lambda x:(jieba.lcut(str(x))).split(","))#apply函数可以有多个参数，第一个是函数，后面是该函数所需用到的变量
#lambda函数 定义了变量“：”后跟的是变量满足的表达式
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.decomposition import NMF
import numpy as np
import pickle 
import warnings  
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

import jieba
import jieba.posseg as pseg
df_orig = pd.read_csv(r'C:\Users\admin\Desktop\sogou.news.50000.csv')#读入文件赋值给df——orig
df_orig.shape#求出该表格矩阵的维数
df_orig.head()#观察数据是否读取正确（head（）函数只读取前5行）
# Check how many rows have missing values for consumer_complaint_narrative column
df_orig.content.isnull().values.sum()
# extract only the relevant column for this project
df_orig = df_orig.loc[:,['content']]#loc函数左边的参数是查看的行的范围，右边是你所要查看的特定的列（就是content

In [5]:
df_orig = pd.read_csv(r'C:\Users\admin\Desktop\sogou.news.50000.csv')#读入文件赋值给df——orig

In [4]:
df_orig.shape#求出该表格矩阵的维数

(49828, 4)

In [5]:
df_orig.head()#观察数据是否读取正确（head（）函数只读取前5行）

,url,docno,contenttitle,content
0,http://gongyi.sohu.com/20120706/n347457739.shtml,98590b972ad2f0ea-34913306c0bb3300,深圳地铁将设立ＶＩＰ头等车厢 买双倍票可享坐票,南都讯 记者刘凡 周昌和 任笑一 继推出日票后，深圳今后将设地铁ＶＩＰ头等车厢，设坐票制。昨...
1,http://gongyi.sohu.com/20120724/n348878190.shtml,5fa7926d2cd2f0ea-34913306c0bb3300,爸爸为女儿百万建幼儿园 消防设施３年仍不过关,NaN
2,http://gongyi.sohu.com/s2008/sourceoflife/,f2467af22cd2f0ea-34913306c0bb3300,中国西部是地球上主要干旱带之一，妇女是当地劳动力．．．,同心县地处宁夏中部干旱带的核心区， 冬寒长，春暖迟，夏热短，秋凉早，干旱少雨，蒸发强烈，风大...
3,http://gongyi.sohu.com/20120612/n345424232.shtml,0dadd5002ed2f0ea-34913306c0bb3300,思源焦点公益基金救助孩子：永康,不满一岁的永康是个饱经病痛折磨的孩子，２０１１年７月５日出生的他，患有先天性心脏病、疝气，一...
4,http://gongyi.sohu.com/20120629/n346847569.shtml,be6bcb252fd2f0ea-34913306c0bb3300,康师傅回应转卖废弃茶叶：下家承诺用废料做枕头,就废弃茶叶被转手事件发声明本报讯（记者刘俊） “我们也是受害者！”昨日，有媒体报道称康师傅...


In [6]:
# Check how many rows have missing values for consumer_complaint_narrative column
df_orig.content.isnull().values.sum()

3639

In [6]:
# extract only the relevant column for this project
df_orig = df_orig.loc[:,['content']]#loc函数左边的参数是查看的行的范围，右边是你所要查看的特定的列（就是content那一列）

In [7]:
# Exclude all rows with null consumer_complaint_narrative
df_orig.dropna(inplace=True)#，用于过滤数据中的缺失数据.drop函数有thresh，inplace用法，inplace=True：不创建新的对象，直接对原始对象进行修改，inplace=False：对数据进行修改，创建并返回新的对象承载其修改结果。
df_orig.shape

(46189, 1)

In [8]:
data=jieba.lcut(str(df_orig['content']))


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 1.782 seconds.
Prefix dict has been built succesfully.


In [9]:
print(data)

['0', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '南', '都', '讯', '\u3000', '记者', '刘凡', '\u3000', '周昌', '和', '\u3000', '任笑', '一', '\u3000', '继', '推出', '日票', '后', '，', '深圳', '今后', '将', '设', '地铁', 'Ｖ', 'Ｉ', 'Ｐ', '头等', '车厢', '，', '设', '坐票', '制', '。', '昨', '...', '\n', '2', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '同心县', '地处', '宁夏', '中部', '干旱', '带', '的', '核心区', '，', '\u3000', '冬寒', '长', '，', '春暖迟', '，', '夏热', '短', '，', '秋凉', '早', '，', '干旱', '少雨', '，', '蒸发', '强烈', '，', '风大', '...', '\n', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '不满', '一岁', '的', '永康', '是', '个', '饱经', '病痛', '折磨', '的', '孩子', '，', '２', '０', '１', '１', '年', '７', '月', '５', '日', '出生', '的', '他', '，', '患有', '先天性', '心脏病', '、', '疝气', '，', '一', '...', '\n', '4', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '就', '废弃', '茶叶', '被', '转手', '事件', '发声明', '\ue40c', '本报讯', '（', '记者', '刘俊', '）', '\u3000', '“', '我们', '也', '是', '受害者', '！', '”', '昨日', '，', '有', '媒体报道', '称', '康师傅', '...', '\n', '5', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '·', '奖励', '办法', '：

In [23]:
# 作业一：为 content 中的内容分词，请替换下行中的 ...
df_orig['content'] = (df_orig['content']).apply(lambda x:" ,".join((jieba.lcut(str(x)))))#apply函数可以有多个参数，第一个是函数，后面是该函数所需用到的变量
#lambda函数 定义了变量“：”后跟的是变量满足的表达式

In [22]:
print((df_orig['content']))

0        南 都 讯 　 记者 刘凡 　 周昌 和 　 任笑 一 　 继 推出 日票 后 ， 深圳 今...
2        同心县 地处 宁夏 中部 干旱 带 的 核心区 ， 　 冬寒 长 ， 春暖迟 ， 夏热 短 ...
3        不满 一岁 的 永康 是 个 饱经 病痛 折磨 的 孩子 ， ２ ０ １ １ 年 ７ 月 ５...
4        就 废弃 茶叶 被 转手 事件 发声明  本报讯 （ 记者 刘俊 ） 　 “ 我们 也 是...
5        · 奖励 办法 ： 率先 提交 的 前 １ ０ ０ 个 创意 项目 ， 经 评估 ， 可 优...
                               ...                        
49823    美国 休斯敦大学 一项 调查 显示 ， 电视 遥控器 和 床 前灯 开关 是 酒店 内 沾染...
49824    本报讯 记者 　 彭敏熠 　 意大利人 没 能 延续 每逢 国内 爆出 足球 丑闻 ， 国家...
49825    本报 ７ 月 ２ ０ 日讯 　 （ 记者 　 李飞飞 ） 今日 ， 一揽子 人事 任命 经 ...
49826    根据 规划 ， 青岛 蓝色 硅谷 “ 一带 一区 一园 ” ， 不仅 能 发挥 青岛 在 海...
49827    本报 ７ 月 ２ ６ 日讯 （ 记者 　 陶小爱 ） 近日 ， 长沙市 天心区 同 发 小额...
Name: content, Length: 46189, dtype: object


In [12]:
# Create a doc id for merging the results back to the original file 
df_orig.insert(0, 'Doc_Id', range(0, 0 + len(df_orig)))#insert不是只有两个参数吗
df_orig.head()

,Doc_Id,content
0,0,南 都 讯 记者 刘凡 周昌 和 任笑 一 继 推出 日票 后 ， 深圳 今...
2,1,同心县 地处 宁夏 中部 干旱 带 的 核心区 ， 冬寒 长 ， 春暖迟 ， 夏热 短 ...
3,2,不满 一岁 的 永康 是 个 饱经 病痛 折磨 的 孩子 ， ２ ０ １ １ 年 ７ 月 ５...
4,3,就 废弃 茶叶 被 转手 事件 发声明  本报讯 （ 记者 刘俊 ） “ 我们 也 是...
5,4,· 奖励 办法 ： 率先 提交 的 前 １ ０ ０ 个 创意 项目 ， 经 评估 ， 可 优...


In [13]:
# save for use later
df_orig.to_pickle('df_orig.pkl')

In [14]:
df = df_orig.loc[:,['content','Doc_Id']]

In [15]:
print(df['content'])

0        南 都 讯 　 记者 刘凡 　 周昌 和 　 任笑 一 　 继 推出 日票 后 ， 深圳 今...
2        同心县 地处 宁夏 中部 干旱 带 的 核心区 ， 　 冬寒 长 ， 春暖迟 ， 夏热 短 ...
3        不满 一岁 的 永康 是 个 饱经 病痛 折磨 的 孩子 ， ２ ０ １ １ 年 ７ 月 ５...
4        就 废弃 茶叶 被 转手 事件 发声明  本报讯 （ 记者 刘俊 ） 　 “ 我们 也 是...
5        · 奖励 办法 ： 率先 提交 的 前 １ ０ ０ 个 创意 项目 ， 经 评估 ， 可 优...
                               ...                        
49823    美国 休斯敦大学 一项 调查 显示 ， 电视 遥控器 和 床 前灯 开关 是 酒店 内 沾染...
49824    本报讯 记者 　 彭敏熠 　 意大利人 没 能 延续 每逢 国内 爆出 足球 丑闻 ， 国家...
49825    本报 ７ 月 ２ ０ 日讯 　 （ 记者 　 李飞飞 ） 今日 ， 一揽子 人事 任命 经 ...
49826    根据 规划 ， 青岛 蓝色 硅谷 “ 一带 一区 一园 ” ， 不仅 能 发挥 青岛 在 海...
49827    本报 ７ 月 ２ ６ 日讯 （ 记者 　 陶小爱 ） 近日 ， 长沙市 天心区 同 发 小额...
Name: content, Length: 46189, dtype: object


In [16]:
# Get the word count for each document
df['word_count'] = df['content'].apply(lambda x: len(str(x)))#新建一个列
df.head()

,content,Doc_Id,word_count
0,南 都 讯 记者 刘凡 周昌 和 任笑 一 继 推出 日票 后 ， 深圳 今...,0,1091
2,同心县 地处 宁夏 中部 干旱 带 的 核心区 ， 冬寒 长 ， 春暖迟 ， 夏热 短 ...,1,249
3,不满 一岁 的 永康 是 个 饱经 病痛 折磨 的 孩子 ， ２ ０ １ １ 年 ７ 月 ５...,2,365
4,就 废弃 茶叶 被 转手 事件 发声明  本报讯 （ 记者 刘俊 ） “ 我们 也 是...,3,1200
5,· 奖励 办法 ： 率先 提交 的 前 １ ０ ０ 个 创意 项目 ， 经 评估 ， 可 优...,4,177


In [51]:
print(df['word_count'])

0        1091
2         249
3         365
4        1200
5         177
         ... 
49823     257
49824     674
49825     266
49826    2042
49827     480
Name: word_count, Length: 46189, dtype: int64


In [52]:
# Summary statistics
df.word_count.describe()

count    46189.000000
mean       768.368140
std        671.239559
min         12.000000
25%        266.000000
50%        553.000000
75%       1082.000000
max       3572.000000
Name: word_count, dtype: float64

In [19]:
# just to get a sizable data to work with in this tutorial
df1= df[(df['word_count']>=293)&(df['word_count']<=412)]

In [54]:
df1.shape

(5109, 3)

In [18]:
df1.head()

NameError: name 'df1' is not defined

In [20]:
# Removing unwanted characters
import re
df1['content'] = df1['content'].str.replace('', '')
df1.head()

,content,Doc_Id,word_count
3,不满 一岁 的 永康 是 个 饱经 病痛 折磨 的 孩子 ， ２ ０ １ １ 年 ７ 月 ５...,2,365
9,２ ０ ０ ９ 年 ８ 月 ， 《 ２ ０ ０ ９ 中国 慈善 导航 行动 》 第一季 正式...,6,295
29,２ ０ ０ ８ 年 ８ 月 的 一个 清晨 ， Ａ 市 女子 某 医院 大院 的 水淹 地里...,23,390
46,瑞丽 阳光 基金 成立 于 ２ ０ ０ ８ 年 １ ２ 月 １ ２ 日 ， 是 《 瑞丽 》...,38,306
57,个人信息 姓名 ＊ ： 性别 ＊ ： 年龄 ＊ ： 教育 程度 ＊ ...,49,382


In [21]:
# Top 20 most frequent words
freq = pd.Series(' '.join(df1['content']).split()).value_counts()[:20]
freq

，    43986
的    22720
。    20707
１    11234
０    10377
２     9828
、     8628
在     7080
７     4914
６     4863
”     4850
“     4848
月     4688
３     4626
了     4595
５     4470
日     4371
和     3998
８     3324
４     3310
dtype: int64

In [58]:
def nouns(text):
    is_noun = lambda pos: pos[0] == 'n'
    pos_tagged = pseg.lcut(text.replace(' ', ''))
    all_nouns = [word for (word, pos) in pos_tagged if is_noun(pos)]
    return ' '.join(all_nouns)

In [59]:
df1['data_nouns'] = pd.DataFrame(df1.content.apply(nouns))
df1.head()

,content,Doc_Id,word_count,data_nouns
3,不满，一岁，的，永康，是，个，饱经，病痛，折磨，的，孩子，，，２，０，１，１，年，７，月，５，日，出生，的，他，，，患有，先天性，心脏病，、，疝气，，，一，出生，便，被遗弃，。，２，０，１，２，年，１，月，８，日，，，才，５，个，月，大，的，永康，被，发现，呼吸困难，，，随后，送往，医院，进行，抢救，治疗，，，病情，稳定，后于，１，月，２，８，日，出院，。，，２，０，１，２，年，２，月，１，３，号，，，永康，在，思源，焦点，公益，基金，的，帮助，下，在，医院，接受，手术，治疗，，，术后，仅，８，天，，，永康，突发，右侧，腹股沟，斜，疝，嵌顿，及，肠梗阻，，，又，再次，进行，抢救，治疗，，，术后，进，重症，监护室，。，３，月，７，日，，，几经，病痛，折磨，的，永康，终于，康复，出院，，，目前，他，的，病情，已经，稳定，。,2,365,永康 病痛 孩子 先天性 心脏病 疝气 被遗弃 永康 医院 病情 出院 永康 思源 焦点 公益 基金 医院 手术 永康 腹股沟 肠梗阻 重症 监护室 病痛 永康 康复 出院 病情
9,２，０，０，９，年，８，月，，，《，２，０，０，９，中国，慈善，导航，行动，》，第一季，正式，启动，，，此档，由，Ｃ，Ｃ，Ｔ，Ｖ，－，１，２，《，大家，看法，》，精心制作，的，节目，将，于，２，０，０，９，年，８，月，１，７，日至，８，月，２，３，日，每晚，８，点整，在，中央电视台，社会，与，法，频道，Ｃ，Ｃ，Ｔ，Ｖ，－，１，２，正式，播出，。，节目，将，从，１，０，０，多个，“，５，．，１，２，灾后，重建，资助，项目，”，中，挑选出，１，８，个，优秀，服务项目，参与，电视，评估，及，展示，，，１，８，个，项目，中，究竟，哪些项目，将，得到，千万，基金，的，支持，…， ，［，详细，］,6,295,中国 慈善 档 大家 节目 中央电视台 社会 频道 节目 灾 资助 项目 服务项目 电视 项目 哪些项目 基金
29,２，０，０，８，年，８，月，的，一个，清晨，，，Ａ，市，女子，某，医院，大院，的，水淹，地里，，，一个，熟睡，的，女婴，被，人，发现，。，出生，仅，两天，的，她，被，父母，遗弃，，，因为，她，患有，先天性，法式，四联，症，，，心脏，动脉，导管，未闭，，，大动脉，异常，。，，随后，女婴，被，福利院，收养，坚强，的，她，安静，地，等待，着，希望，的，出现，。，，２，０，１，２，年，１，月，，，萍萍，来到，北京，进行，治疗，。，２，０，１，２，年，３，月，２，３，日，，，在，思源，焦点，公益，基金，的，资助，下，，，萍萍，在，医院，进行，了，近，１，０，小时，的，高强度，手术，。，，虽然，手术，是，成功，的，，，但，术后，萍萍，并，没有，完全，摆脱，危险，，，她，的，心脏，只能，借助，体外循环，才能，跳动，。，目前，，，坚强，的，萍萍，仍，在，重症，病房，接受，监护，。,23,390,市 女子 医院 大院 水淹 女婴 人 父母 先天性 法式 四联 心脏 动脉 导管 大动脉 女婴 福利院 安静 萍 萍 北京 思源 焦点 公益 基金 资助 萍 萍 医院 小时 高强度 手术 手术 萍 萍 心脏 体外循环 萍 萍 重症 病房
46,瑞丽，阳光，基金，成立，于，２，０，０，８，年，１，２，月，１，２，日，，，是，《，瑞丽，》，杂志社，与，中国扶贫基金会，合作，成立，的，专项基金，，，专注，于，服务，中国，弱势，儿童，群体，。，，瑞丽，阳光，基金，成立，两年，以来，，，已，陆续，在，河北，、，青海，、，四川，等，省，六个，国家级，贫困地区，启动，了，系列，关爱，儿童，的，援助，计划，。，，《，瑞丽，》，杂志社，作为，一个，负责，任，的，企业，公民，，，将，有效，利用，“，瑞丽，阳光，基金，”，和，媒体，品牌，影响力，，，为，改善，中国，弱势，儿童，的，生存环境，和，推动，社会，慈善，价值观，的，形成，尽，一份，力量，。，［，详细，］,38,306,瑞丽 阳光 基金 瑞丽 杂志社 中国扶贫基金会 专项基金 中国 弱势 儿童 群体 瑞丽 阳光 基金 河北 青海 四川 省 贫困地区 儿童 计划 瑞丽 杂志社 企业 公民 利用 瑞丽 阳光 基金 媒体 品牌 影响力 中国 弱势 儿童 社会 慈善 价值观 力量
57,个人信息，，姓名，＊， ，：，，性别，＊， ，：，，年龄，＊， ，：，，教育，程度，＊， ，：，，职业，＊， ，：，，手机，＊， ，：，，电子邮箱，＊， ，：，，Ｑ，Ｑ，／，Ｍ，Ｓ，Ｎ，号，：，通信地址，＊， ，：，，邮编，＊， ，：，，特长，或，技能，＊， ，：，，参加，其他，公益，组织，或，活动，情况， ，：，，愿意，参加，的，志愿，服务，（，请画，勾，，，可多选，），，宣传，发动，工作， ，学校，考察，与，调研，活动，，鞋子，的，发放，工作， ，有关，的，后续，工作，，志愿者，招募，和，管理工作， ，其他，，志愿，服务，时间，选择，（，请画，勾，，，可多选，），，工作日，的，上午， ，工作日，的，下午，，工作日，的，晚上， ，周六，／，日，的，上午，，周六，／，日，的，下午， ，周六，／，日，的，晚上，，其它,49,382,个人信息 姓名 性别 年龄 程度 职业 手机 电子邮箱 通信地址 邮编 特长 技能 公益 情况 志愿 学校 鞋子 志愿者 管理工作 志愿 时间 工作日 工作日 工作日


In [60]:
# Further cleaning, removing stopwords
import re
temp =[]
my_stop_words = open(r'C:\Users\admin\Desktop\stop-words.txt').read().split('\n')

for sentence in df1['data_nouns']:
    words = [word for word in sentence.split(' ') if word not in my_stop_words]
    temp.append(words)
    
final_X = temp

In [61]:
final_X[:2]

[['永康',
  '病痛',
  '孩子',
  '先天性',
  '心脏病',
  '疝气',
  '被遗弃',
  '永康',
  '医院',
  '病情',
  '出院',
  '永康',
  '思源',
  '焦点',
  '公益',
  '基金',
  '医院',
  '手术',
  '永康',
  '腹股沟',
  '肠梗阻',
  '重症',
  '监护室',
  '病痛',
  '永康',
  '康复',
  '出院',
  '病情'],
 ['中国',
  '慈善',
  '档',
  '大家',
  '节目',
  '中央电视台',
  '社会',
  '频道',
  '节目',
  '灾',
  '资助',
  '项目',
  '服务项目',
  '电视',
  '项目',
  '哪些项目',
  '基金']]

In [62]:
sent = []
for row in final_X:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)
final_X = sent

In [63]:
print(final_X[:2])

[' 永康 病痛 孩子 先天性 心脏病 疝气 被遗弃 永康 医院 病情 出院 永康 思源 焦点 公益 基金 医院 手术 永康 腹股沟 肠梗阻 重症 监护室 病痛 永康 康复 出院 病情', ' 中国 慈善 档 大家 节目 中央电视台 社会 频道 节目 灾 资助 项目 服务项目 电视 项目 哪些项目 基金']


In [64]:
# remove unwanted characters, numbers and symbols 
df1['cleaned'] = final_X
df1.head()

,content,Doc_Id,word_count,data_nouns,cleaned
3,不满，一岁，的，永康，是，个，饱经，病痛，折磨，的，孩子，，，２，０，１，１，年，７，月，５，日，出生，的，他，，，患有，先天性，心脏病，、，疝气，，，一，出生，便，被遗弃，。，２，０，１，２，年，１，月，８，日，，，才，５，个，月，大，的，永康，被，发现，呼吸困难，，，随后，送往，医院，进行，抢救，治疗，，，病情，稳定，后于，１，月，２，８，日，出院，。，，２，０，１，２，年，２，月，１，３，号，，，永康，在，思源，焦点，公益，基金，的，帮助，下，在，医院，接受，手术，治疗，，，术后，仅，８，天，，，永康，突发，右侧，腹股沟，斜，疝，嵌顿，及，肠梗阻，，，又，再次，进行，抢救，治疗，，，术后，进，重症，监护室，。，３，月，７，日，，，几经，病痛，折磨，的，永康，终于，康复，出院，，，目前，他，的，病情，已经，稳定，。,2,365,永康 病痛 孩子 先天性 心脏病 疝气 被遗弃 永康 医院 病情 出院 永康 思源 焦点 公益 基金 医院 手术 永康 腹股沟 肠梗阻 重症 监护室 病痛 永康 康复 出院 病情,永康 病痛 孩子 先天性 心脏病 疝气 被遗弃 永康 医院 病情 出院 永康 思源 焦点 公益 基金 医院 手术 永康 腹股沟 肠梗阻 重症 监护室 病痛 永康 康复 出院 病情
9,２，０，０，９，年，８，月，，，《，２，０，０，９，中国，慈善，导航，行动，》，第一季，正式，启动，，，此档，由，Ｃ，Ｃ，Ｔ，Ｖ，－，１，２，《，大家，看法，》，精心制作，的，节目，将，于，２，０，０，９，年，８，月，１，７，日至，８，月，２，３，日，每晚，８，点整，在，中央电视台，社会，与，法，频道，Ｃ，Ｃ，Ｔ，Ｖ，－，１，２，正式，播出，。，节目，将，从，１，０，０，多个，“，５，．，１，２，灾后，重建，资助，项目，”，中，挑选出，１，８，个，优秀，服务项目，参与，电视，评估，及，展示，，，１，８，个，项目，中，究竟，哪些项目，将，得到，千万，基金，的，支持，…， ，［，详细，］,6,295,中国 慈善 档 大家 节目 中央电视台 社会 频道 节目 灾 资助 项目 服务项目 电视 项目 哪些项目 基金,中国 慈善 档 大家 节目 中央电视台 社会 频道 节目 灾 资助 项目 服务项目 电视 项目 哪些项目 基金
29,２，０，０，８，年，８，月，的，一个，清晨，，，Ａ，市，女子，某，医院，大院，的，水淹，地里，，，一个，熟睡，的，女婴，被，人，发现，。，出生，仅，两天，的，她，被，父母，遗弃，，，因为，她，患有，先天性，法式，四联，症，，，心脏，动脉，导管，未闭，，，大动脉，异常，。，，随后，女婴，被，福利院，收养，坚强，的，她，安静，地，等待，着，希望，的，出现，。，，２，０，１，２，年，１，月，，，萍萍，来到，北京，进行，治疗，。，２，０，１，２，年，３，月，２，３，日，，，在，思源，焦点，公益，基金，的，资助，下，，，萍萍，在，医院，进行，了，近，１，０，小时，的，高强度，手术，。，，虽然，手术，是，成功，的，，，但，术后，萍萍，并，没有，完全，摆脱，危险，，，她，的，心脏，只能，借助，体外循环，才能，跳动，。，目前，，，坚强，的，萍萍，仍，在，重症，病房，接受，监护，。,23,390,市 女子 医院 大院 水淹 女婴 人 父母 先天性 法式 四联 心脏 动脉 导管 大动脉 女婴 福利院 安静 萍 萍 北京 思源 焦点 公益 基金 资助 萍 萍 医院 小时 高强度 手术 手术 萍 萍 心脏 体外循环 萍 萍 重症 病房,市 女子 医院 大院 水淹 女婴 人 父母 先天性 法式 四联 心脏 动脉 导管 大动脉 女婴 福利院 安静 萍 萍 北京 思源 焦点 公益 基金 资助 萍 萍 医院 小时 高强度 手术 手术 萍 萍 心脏 体外循环 萍 萍 重症 病房
46,瑞丽，阳光，基金，成立，于，２，０，０，８，年，１，２，月，１，２，日，，，是，《，瑞丽，》，杂志社，与，中国扶贫基金会，合作，成立，的，专项基金，，，专注，于，服务，中国，弱势，儿童，群体，。，，瑞丽，阳光，基金，成立，两年，以来，，，已，陆续，在，河北，、，青海，、，四川，等，省，六个，国家级，贫困地区，启动，了，系列，关爱，儿童，的，援助，计划，。，，《，瑞丽，》，杂志社，作为，一个，负责，任，的，企业，公民，，，将，有效，利用，“，瑞丽，阳光，基金，”，和，媒体，品牌，影响力，，，为，改善，中国，弱势，儿童，的，生存环境，和，推动，社会，慈善，价值观，的，形成，尽，一份，力量，。，［，详细，］,38,306,瑞丽 阳光 基金 瑞丽 杂志社 中国扶贫基金会 专项基金 中国 弱势 儿童 群体 瑞丽 阳光 基金 河北 青海 四川 省 贫困地区 儿童 计划 瑞丽 杂志社 企业 公民 利用 瑞丽 阳光 基金 媒体 品牌 影响力 中国 弱势 儿童 社会 慈善 价值观 力量,瑞丽 阳光 基金 瑞丽 杂志社 中国扶贫基金会 专项基金 中国 弱势 儿童 群体 瑞丽 阳光 基金 河北 青海 四川 省 贫困地区 儿童 计划 瑞丽 杂志社 企业 公民 利用 瑞丽 阳光 基金 媒体 品牌 影响力 中国 弱势 儿童 社会 慈善 价值观 力量
57,个人信息，，姓名，＊， ，：，，性别，＊， ，：，，年龄，＊， ，：，，教育，程度，＊， ，：，，职业，＊， ，：，，手机，＊， ，：，，电子邮箱，＊， ，：，，Ｑ，Ｑ，／，Ｍ，Ｓ，Ｎ，号，：，通信地址，＊， ，：，，邮编，＊， ，：，，特长，或，技能，＊， ，：，，参加，其他，公益，组织，或，活动，情况， ，：，，愿意，参加，的，志愿，服务，（，请画，勾，，，可多选，），，宣传，发动，工作， ，学校，考察，与，调研，活动，，鞋子，的，发放，工作， ，有关，的，后续，工作，，志愿者，招募，和，管理工作， ，其他，，志愿，服务，时间，选择，（，请画，勾，，，可多选，），，工作日，的，上午， ，工作日，的，下午，，工作日，的，晚上， ，周六，／，日，的，上午，，周六，／，日，的，下午， ，周六，／，日，的，晚上，，其它,49,382,个人信息 姓名 性别 年龄 程度 职业 手机 电子邮箱 通信地址 邮编 特长 技能 公益 情况 志愿 学校 鞋子 志愿者 管理工作 志愿 时间 工作日 工作日 工作日,个人信息 姓名 性别 年龄 程度 职业 手机 电子邮箱 通信地址 邮编 特长 技能 公益 情况 志愿 学校 鞋子 志愿者 管理工作 志愿 时间 工作日 工作日 工作日


In [65]:
# save for use later
df1.to_pickle('data_prep.pkl')

In [ ]:
#df1 = pd.read_pickle(data_prep.pkl)